# Visual perception for autonomous vehicles using active learning

Training computer vision models for autonomous driving to achieve high-end performance requires large labeled datasets, which can be prohibitively expensive. This notebook shows an end to end pipeline that streamlines the process of labelling driving scene datasets for a task, using SageMaker Groundtruth autolabelling and active learning. The task that we focus is pedestrian detection in camera images. This is framed as an object detection task where the goal of the final trained model is to predict 2D bounding boxes around pedestrians in an image.

The active learning pipeline ensures that, starting with just a handful of a labels we can train a model, predict bounding boxes, compare the models predictions to a real labels and only add more labels if we need to increase the performance of the model. Using Step Functions, we can create a workflow that automates this process and iteratively performs the active learning loop.

In [ ]:
!bash setup.sh

import sagemaker
from package import config, manifest

role = config.role
sess = sagemaker.Session()

## Dataset

The image dataset we are using for this demonstration is a subset of the [A2D2 dataset](https://www.a2d2.audi/a2d2/en/dataset.html). We select a subset of camera frames from the original dataset and have generated 2D bounding box labels around objects of interests from the segmentation maps in the original dataset. 

For flexibility and ease of use, we will be working with an S3 manifest file of the dataset. You can replace the `data_manifest_file` with an s3 or local path to your custom manifest file in the same format to use your own dataset.

In [ ]:
data_manifest_file = 's3://{}/{}/{}'.format(config.solution_upstream_bucket,
                                            config.solution_name,
                                            'data/manifests/a2d2_visual_perception_bbox.manifest')

Let's take a peek at single entry in the manifest file

In [ ]:
manifest_rows = manifest.get_manifest_rows_from_path(data_manifest_file)
manifest_rows[0]

Let's try and visualize some images that have annotations

In [ ]:
manifest.visualize_manifest_images(data_manifest_file)

## SageMaker Groundtruth Labeling Workteam

Now that we have seen what data we will be working with. Let's prepare a SageMaker Groundtruth private workteam that we will use as the workforce for data labeling. For the purposes of this demonstration, you will add yourself to a SageMaker private workteam that's managed by a Cognito User Pool. Then when we create labeling jobs during the active learning pipeline, the jobs will be sent to this workteam and you'll be able to go and manually label images using the SageMaker Groundtruth labeling UI

### Create a work team

The first step is to create a workteam. By launching this solution the necessary Cognito resources for create a new private workteam should have already been created. If you have already used GroundTruth before and set up a Private Workforce, the new workteam will be created against the existing Groundtruth Cognito configuration. However, if there are no existing Cognito configurations for labeling jobs, we will used the Cognito resources created when launching the solution.

In [ ]:
from package import workteam

cognito_config = workteam.get_cognito_configs()
workteam_name = config.solution_prefix + "-workteam"

if cognito_config is None:
    workteam_arn = workteam.create_groundtruth_workteam(workteam_name, config)
    userpool = config.cognito_user_pool
else:
    workteam_arn = workteam.create_groundtruth_workteam(workteam_name, cognito_config)
    userpool = cognito_config['UserPool']

### Update your user invitation email with the new workteam

Now your private groundtruth workteam is ready to go. Now we'll setup the domain for the Groundtruth labeling jobs that and also update the backing Cognito user pool with this domain. You'll be able to use this to invite people to help you label your dataset.

In [ ]:
signup_domain = workteam.get_signup_domain(workteam_name=workteam_name)

workteam.update_user_pool_with_invite(userpool, signup_domain)

### Invite yourself and others to label your dataset

Invite people to help you label your dataset. Use the following link to enter your email
address to get an invite.


In [ ]:
"https://{}.console.aws.amazon.com/sagemaker/groundtruth?region={}#/labeling-workforces/add-workers".format(config.region, config.region)

Then open the following link in a new tab to add your email address to the workteam. 

Click on the **Workers** tab, then click add worker to add your email address.

In [ ]:
"https://{}.console.aws.amazon.com/sagemaker/groundtruth?region={}#/labeling-workforces/private-details/{}".format(config.region, config.region, workteam_name)

## Prepare Dataset for Active Learning

Now, we will prepare the dataset for the active learning loop. For this, we start with a manifest file that is partially labeled. Starting with this allows us to have the first stage of the Active Learning loop be a model training job instead of a processing job. We could also start with a fully unlabeled dataset but this would mean the first stage of our active learning loop would be a labeling job. 

### Upload partially labeled input to S3

The utility function below will copy the `data_manifest_file` into an input manifest for this experiment. In the input manifest, we will only keep bounding box labels for a subset of the data, and exclude other class labels besides Pedestrians before uploading the input manifest to the s3 path in `input_data`.

To see the input manifest, you can open the file `./manifests/partially_labeled_input.manifest`

In [ ]:
from package.active_learning import prepare

input_data = 's3://{}/{}/{}'.format(config.solution_bucket, 'active-learning', config.s3_data_prefix)
s3_input_manifest_path = prepare.partially_labeled_input(input_data, manifest_rows)

Next, we will prepare the job template and class_labels that will be used by all GroundTruth labeling job in the Active Learning loop. You can see the contents of the template at `./artifacts/instructions.template` and the simple class labels for Pedestrian bounding box at `./artifacts/class_labels.json` 

In [ ]:
from package.active_learning import prepare

s3_class_labels_path, s3_template_path = prepare.labels_config_and_template(input_data)

Now that we have our input data artifacts in the s3 input location, it is time to create the input request to the active learning loop

### Create Groundtruth request for active learning 

The input request to the active learning loop will be in the form of the [GroundTruth Labeling Job request syntax](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateLabelingJob.html) but this will be passed as in input to the Step Functions StateMachine for the active learning loop. The stateMachine will be able to parse the request and select the appropriate state of the active learning loop to begin execution.

You can see the input request at the path `./requests/ground_truth.request`

In [ ]:
from package.active_learning import request

gt_request = request.create_ground_truth_request(s3_input_manifest_path,
                                                 s3_class_labels_path,
                                                 s3_template_path,
                                                 config.region,
                                                 role,
                                                 config.solution_prefix,
                                                 's3://{}/{}/output/'.format(config.solution_bucket, config.s3_data_prefix),
                                                 workteam_arn=workteam_arn)

## Start Active Learning Loop

All the input configurations are complete so we can create a new instance of the active learning pipeline and start execution. As mentioned already, we are using a Step Functions workflow to orchestrate the active learning loop.

In [ ]:
from package.active_learning.step_functions import ActiveLearningPipeline

step_functions_pipeline = ActiveLearningPipeline(config.step_functions_active_learning)

In [ ]:
execution_arn = step_functions_pipeline.start_execution(gt_request)